Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Step %d - Loss %f - Minibatch %.1f%% - Validation %.1f%%' %
                  (step, l, accuracy(predictions, batch_labels),accuracy(
                    valid_prediction.eval(), valid_labels)))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Step 0 - Loss 2.534234 - Minibatch 18.8% - Validation 11.1%
Step 50 - Loss 1.474175 - Minibatch 50.0% - Validation 50.1%
Step 100 - Loss 0.775141 - Minibatch 81.2% - Validation 73.2%
Step 150 - Loss 0.863082 - Minibatch 68.8% - Validation 74.9%
Step 200 - Loss 1.171659 - Minibatch 75.0% - Validation 78.3%
Step 250 - Loss 0.956151 - Minibatch 75.0% - Validation 76.8%
Step 300 - Loss 0.832626 - Minibatch 68.8% - Validation 78.8%
Step 350 - Loss 1.001116 - Minibatch 75.0% - Validation 79.2%
Step 400 - Loss 0.715934 - Minibatch 87.5% - Validation 81.1%
Step 450 - Loss 0.728667 - Minibatch 75.0% - Validation 76.1%
Step 500 - Loss 0.778035 - Minibatch 75.0% - Validation 81.1%
Step 550 - Loss 0.738171 - Minibatch 75.0% - Validation 81.5%
Step 600 - Loss 1.016576 - Minibatch 62.5% - Validation 80.7%
Step 650 - Loss 0.951612 - Minibatch 68.8% - Validation 82.3%
Step 700 - Loss 0.267146 - Minibatch 87.5% - Validation 81.4%
Step 750 - Loss 0.629344 - Minibatch 81.2% - Validation 82.0%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [12]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        # layer 1 convo. max_pool 2x2
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + layer1_biases)
        # layer 2 convo. max_pool 2x2
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + layer2_biases)
        # layer 3 fully connected.
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size)b % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Step %d - Loss %f - Minibatch %.1f%% - Validation %.1f%%' %
                  (step, l, accuracy(predictions, batch_labels),accuracy(
                    valid_prediction.eval(), valid_labels)))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Step 0 - Loss 3.287148 - Minibatch 6.2% - Validation 10.1%
Step 50 - Loss 1.270621 - Minibatch 62.5% - Validation 42.4%
Step 100 - Loss 0.914505 - Minibatch 81.2% - Validation 74.7%
Step 150 - Loss 0.958688 - Minibatch 68.8% - Validation 76.1%
Step 200 - Loss 1.037660 - Minibatch 75.0% - Validation 77.6%
Step 250 - Loss 1.005606 - Minibatch 75.0% - Validation 77.7%
Step 300 - Loss 0.861946 - Minibatch 68.8% - Validation 79.6%
Step 350 - Loss 1.015530 - Minibatch 68.8% - Validation 76.6%
Step 400 - Loss 0.662101 - Minibatch 75.0% - Validation 81.4%
Step 450 - Loss 0.697596 - Minibatch 87.5% - Validation 79.0%
Step 500 - Loss 0.629703 - Minibatch 81.2% - Validation 82.0%
Step 550 - Loss 0.636615 - Minibatch 75.0% - Validation 81.8%
Step 600 - Loss 0.932539 - Minibatch 81.2% - Validation 81.4%
Step 650 - Loss 0.902850 - Minibatch 75.0% - Validation 82.4%
Step 700 - Loss 0.134424 - Minibatch 100.0% - Validation 83.8%
Step 750 - Loss 0.446201 - Minibatch 81.2% - Validation 83.8%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [5]:
import datetime

def eta(start, n, total):
    now = datetime.datetime.now()
    diff = now - start
    secs = (total-n) * 1.0 * diff.seconds / (n+1) # +1 to avoid zero division.
    ends = now + datetime.timedelta(seconds=secs)
    return ends.strftime("%H:%M:%S")


In [41]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

batch_size = 16
patch_size = 5
depth = 16
num_hidden_full_1 = 512
num_hidden_full_2 = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden_full_1], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_full_1]))
    keep3 = tf.placeholder("float")
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden_full_1, num_hidden_full_2], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_full_2]))
    keep4 = tf.placeholder("float")
    layer5_weights = tf.Variable(tf.truncated_normal(
        [num_hidden_full_2, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model. using elu not relu.
    def model(data):
        # layer 1 convo. max_pool 2x2.
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.elu(pool + layer1_biases)
        # layer 2 convo. max_pool 2x2
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.elu(pool + layer2_biases)
        # layer 3 fully connected.
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.elu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep3)
        # layer 4 fully connected
        hidden = tf.nn.elu(tf.matmul(hidden, layer4_weights) + layer4_biases)
        hidden = tf.nn.dropout(hidden, keep4)
        # layer 5 output
        output = tf.matmul(hidden, layer5_weights) + layer5_biases
        return output
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    # optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [43]:
num_steps = 40001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    start = datetime.datetime.now()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                    keep3:0.9, keep4:0.9}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            ends = eta(start, step, num_steps)
            valpred = valid_prediction.eval(feed_dict={keep3:1.0, keep4:1.0})            
            print('Step %d - Loss %f - Minibatch %.1f%% - Validation %.1f%% - ETA %s' %
                  (step, l, accuracy(predictions, batch_labels), accuracy(valpred, valid_labels), ends))
    print('Test accuracy: %.1f%%' %
          accuracy(test_prediction.eval(feed_dict={keep3:1.0, keep4:1.0}), test_labels))

Initialized
Step 0 - Loss 4.270369 - Minibatch 0.0% - Validation 13.9% - ETA 18:30:25
Step 500 - Loss 1.077539 - Minibatch 62.5% - Validation 77.7% - ETA 19:43:37
Step 1000 - Loss 0.722461 - Minibatch 87.5% - Validation 79.6% - ETA 19:39:01
Step 1500 - Loss 0.530717 - Minibatch 81.2% - Validation 81.4% - ETA 19:39:16
Step 2000 - Loss 1.372873 - Minibatch 50.0% - Validation 82.4% - ETA 19:38:43
Step 2500 - Loss 1.182326 - Minibatch 62.5% - Validation 81.9% - ETA 19:38:40
Step 3000 - Loss 0.239541 - Minibatch 100.0% - Validation 83.1% - ETA 19:37:57
Step 3500 - Loss 0.815364 - Minibatch 75.0% - Validation 83.2% - ETA 19:38:36
Step 4000 - Loss 0.672563 - Minibatch 81.2% - Validation 84.2% - ETA 19:39:35
Step 4500 - Loss 0.540213 - Minibatch 93.8% - Validation 84.6% - ETA 19:40:11
Step 5000 - Loss 0.417364 - Minibatch 87.5% - Validation 84.7% - ETA 19:40:49
Step 5500 - Loss 0.621419 - Minibatch 75.0% - Validation 85.2% - ETA 19:40:29
Step 6000 - Loss 0.794168 - Minibatch 62.5% - Validation

In [6]:
# DL with inception.

from tensorflow.python.framework import ops
ops.reset_default_graph()

batch_size = 16
patch_size = 5
depth = 16
num_hidden_full_1 = 512
num_hidden_full_2 = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # standard conv2d:
    # layer3_weights = tf.Variable(tf.truncated_normal(
    #     [image_size // 4 * image_size // 4 * depth, num_hidden_full_1], stddev=0.1))
    # inception is = [16, 28, 28, 64] reshaped (16, 50176)
    layer3_weights = tf.Variable(tf.truncated_normal(
        [50176, num_hidden_full_1], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_full_1]))
    keep3 = tf.placeholder("float")
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden_full_1, num_hidden_full_2], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_full_2]))
    keep4 = tf.placeholder("float")
    layer5_weights = tf.Variable(tf.truncated_normal(
        [num_hidden_full_2, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    # vars for inception
    inception_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, num_channels, depth], stddev=0.1))
    inception_1x1_biases = tf.Variable(tf.zeros([depth]))
    pre_inception_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, num_channels, depth], stddev=0.1))
    pre_inception_1x1_biases = tf.Variable(tf.zeros([depth]))
    inception_1x1_pool_weights = tf.Variable(tf.truncated_normal(
        [1, 1, num_channels, depth], stddev=0.1))
    inception_1x1_pool_biases = tf.Variable(tf.zeros([depth]))
    inception_3x3_weights = tf.Variable(tf.truncated_normal(
        [3, 3, depth, depth], stddev=0.1))
    inception_3x3_biases = tf.Variable(tf.zeros([depth]))
    inception_5x5_weights = tf.Variable(tf.truncated_normal(
        [5, 5, depth, depth], stddev=0.1))
    inception_5x5_biases = tf.Variable(tf.zeros([depth]))

    def inception_layer(data):
        # Inception 1x1
        conv_1x1 = tf.nn.conv2d(data, inception_1x1_weights, [1, 1, 1, 1], padding='SAME')
        conv_1x1 = tf.nn.relu(conv_1x1 + inception_1x1_biases)
        ## 1x1 - before the bigger patches
        conv_pre = tf.nn.conv2d(data, pre_inception_1x1_weights, [1, 1, 1, 1], padding='SAME')
        conv_pre = tf.nn.relu(conv_pre + pre_inception_1x1_biases)
        # Pooling 3x3
        ## average pool followed by a 1x1
        conv_pool = tf.nn.avg_pool(data, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME')
        conv_pool = tf.nn.conv2d(conv_pool, inception_1x1_pool_weights, [1, 1, 1, 1], padding='SAME')
        conv_pool = tf.nn.relu(conv_pool + inception_1x1_pool_biases)
        # Inception 3x3
        ## 1x1 followed by a 3x3
        conv_3x3 = tf.nn.conv2d(conv_pre, inception_3x3_weights, [1, 1, 1, 1], padding='SAME')
        conv_3x3 = tf.nn.relu(conv_3x3 + inception_3x3_biases)
        # Inception 5x5
        ## 1x1 followed by a 5x5
        conv_5x5 = tf.nn.conv2d(conv_pre, inception_5x5_weights, [1, 1, 1, 1], padding='SAME')
        conv_5x5 = tf.nn.relu(conv_5x5 + inception_5x5_biases)
        inception_result = tf.concat(3, [conv_1x1, conv_3x3, conv_5x5, conv_pool])
        return inception_result

    # Model. using elu not relu.
    def model(data):
        # layer 1 convo. max_pool 2x2.
        #conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        #pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        #hidden = tf.nn.elu(pool + layer1_biases)
        # layer 2 convo. max_pool 2x2
        #conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        #pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        #hidden = tf.nn.elu(pool + layer2_biases)
        hidden = inception_layer(data)
        # layer 3 fully connected.
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.elu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep3)
        # layer 4 fully connected
        hidden = tf.nn.elu(tf.matmul(hidden, layer4_weights) + layer4_biases)
        hidden = tf.nn.dropout(hidden, keep4)
        # layer 5 output
        output = tf.matmul(hidden, layer5_weights) + layer5_biases
        return output
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    # optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 10001 # 40001 IMPOSIBLE ON MY NOTEBOOK.

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    start = datetime.datetime.now()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                    keep3:0.9, keep4:0.9}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            ends = eta(start, step, num_steps)
            valpred = valid_prediction.eval(feed_dict={keep3:1.0, keep4:1.0})            
            print('Step %d - Loss %f - Minibatch %.1f%% - Validation %.1f%% - ETA %s' %
                  (step, l, accuracy(predictions, batch_labels), accuracy(valpred, valid_labels), ends))
    print('Test accuracy: %.1f%%' %
          accuracy(test_prediction.eval(feed_dict={keep3:1.0, keep4:1.0}), test_labels))
